In [ ]:
import numpy as np
import openmdao.api as om

# Define OpenMDAO Components
We'll stick to using `Explicit Component`s for now

In [ ]:
class RangeBreguet(om.ExplicitComponent):
    """This is a simple component to compute the range of an aircraft using the Range Breguet equation."""
    
    def setup(self):
        self.add_input(
            name="l_over_d",
            val=10.0,
            desc="The Lift-over-Drag parameter",
        )
        self.add_input(
            name="speed",
            val=320.0,
            units="knot",
            desc="Speed of the aircraft",
        )
        self.add_input(
            name="sfc",
            val=0.85,
            units="1/h",
            desc="Specific Fuel Consumption",
        )
        self.add_input(
            name="weight_ratio",
            val=0.75,
            desc="The weight ratio between the start and end of the segment",
        )

        self.add_output(
            name="range",
            units="nmi",
            desc="The distance flown by the aircraft in the segment",
        )

    def compute(self, inputs, outputs):
        outputs["range"] = (
            inputs["speed"]
            * inputs["l_over_d"]
            * -np.log(inputs["weight_ratio"])
            / inputs["sfc"]
        )

In [ ]:
class SquadronRange(om.ExplicitComponent):
    def setup(self):
        self.add_input(
            name="aircraft_ranges",
            val=0.0,
            units="nmi",
            shape_by_conn=True,
            desc="The range of the various aircraft in the squadron",
        )
        self.add_output(
            name="squadron_range",
            units="nmi",
            desc="The maximum distance that the squadron can fly",
        )

    def compute(self, inputs, outputs):
        outputs["squadron_range"] = np.min(inputs["aircraft_ranges"])

## Instantiate it and Run it

In [ ]:
rb = RangeBreguet()
rb.setup()

inputs={
    name: data["value"]
    for name, data in rb._static_var_rel2meta.items()
}
outputs = {}

rb.compute(inputs, outputs)
outputs["range"]

# Define OpenMDAO Problem and its Components

In [ ]:
# Faux instances, will be refined based on what comes from SysML2
instances = [
    {
        "id": i + 1,
        "name": f"aircraft_{i + 1}",
    }
    for i in range(10)
]

In [ ]:
prob = om.Problem()
model = prob.model

# Add the Range Breguet equation components per aircraft instance
aircraft_range_comps = [
    model.add_subsystem(
        name=f"""{instance["name"]}_range_breguet""",
        subsys=RangeBreguet(),
        # promotes_inputs=["*"],
    ) for instance in instances
]

# Add a mux'er to merge inputs from instances
mux_comp = model.add_subsystem(
    name="aircraft_ranges",
    subsys=om.MuxComp(
        vec_size=len(instances),
    ),
)
mux_comp.add_var("range", shape=(1,), axis=1, units="nmi")

# Note: this does not work, as minimum takes only two inputs...
# Add the squadron range calculation
# squadron_range = model.add_subsystem(
#     "squadron_range",
#     subsys=om.ExecComp(
#         "squadron_range=minimum(aircraft_ranges)",
#         squadron_range={"units": "nmi"},
#         aircraft_ranges={"units": "nmi", "shape": 10},
#     ),
# )
squadron_range = model.add_subsystem(
    "squadron_range",
    subsys=SquadronRange(),
)

# Connect Variables

In [ ]:
for idx, instance in enumerate(instances):
    model.connect(
        f"""aircraft_{instance["id"]}_range_breguet.range""",
        f"""aircraft_ranges.range_{idx}""",
    )

model.connect(
    "aircraft_ranges.range",
    "squadron_range.aircraft_ranges",
)

# Add an optimizer and define optimization problem

In [ ]:
# prob.driver = om.ScipyOptimizeDriver()
# prob.driver.options["optimizer"] = "COBYLA"

prob.driver = om.SimpleGADriver()

var_ranges = dict(
    l_over_d=(2., 18.),
    speed=(100., 200.),
    sfc=(0.55, 1.2),
    weight_ratio=(0.5, 0.95),
)
bits_by_var = {}
DEFAULT_BITS = 12

for instance in instances:
    component_name = f"""aircraft_{instance["id"]}_range_breguet"""
    for var_name, (lower, upper) in var_ranges.items():
        full_var_name = f"{component_name}.{var_name}"
        model.add_design_var(full_var_name, lower, upper)
        bits_by_var[full_var_name] = DEFAULT_BITS

model.add_objective("squadron_range.squadron_range", scaler=-1.0)

## Configure the Genetic Algorithm Driver

In [ ]:
prob.driver.options["gray"] = True
prob.driver.options["elitism"] = True
prob.driver.options["pop_size"] = 200
prob.driver.options["max_gen"] = 100
prob.driver.options["debug_print"] = [
    # "objs",  # to print objectives values
    # "desvars",  # to print the design variables, but it is too much
]  

# TODO: figure out a way to handle this in the future for these types of drivers?
prob.driver.options["bits"] = bits_by_var

## Add a recorder

In [ ]:
recorder = om.SqliteRecorder('cases.sql')

prob.add_recorder(recorder)

In [ ]:
# Setup the problem to run
prob.setup()

In [ ]:
prob.driver.add_recorder(recorder)
prob.set_solver_print(0)

## We can visualize the $n^2$ diagram

In [ ]:
from openmdao.visualization.n2_viewer.n2_viewer import n2

n2(prob)

In [ ]:
prob.run_model()

In [ ]:
prob["squadron_range.squadron_range"]

In [ ]:
prob.run_driver()
prob.record("final_state")
prob.cleanup()

In [ ]:
prob["squadron_range.aircraft_ranges"], prob["squadron_range.squadron_range"]

In [ ]:
# Instantiate your CaseReader
cr = om.CaseReader("cases.sql")

# List driver cases (do not recurse to system/solver cases, suppress display)
driver_cases = cr.list_cases('driver', recurse=False, out_stream=None)

In [ ]:
# Plot the path the design variables took to convergence
# Note that there are two lines in the right plot because "Z"
# contains two variables that are being optimized
var_x = "aircraft_1_range_breguet.speed"
objective = "squadron_range.squadron_range"

best, average = [], []

all_cases = cr.get_cases()
pop_size = prob.driver.options["pop_size"]
for i in range(int(len(all_cases) / pop_size)):
    pop_values = [
        case[objective]
        for case in all_cases[i*pop_size:(i+1)*pop_size]
    ]
    best.append(np.max(pop_values))
    average.append(np.mean(pop_values))

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)

ax1.plot(np.arange(len(best)), np.array(best))
ax1.set(xlabel="Population", ylabel="Best Value", title="Optimization History")
ax1.grid()

ax2.plot(np.arange(len(average)), np.array(average))
ax2.set(xlabel="Population", ylabel="average Value", title="Optimization History")
ax2.grid()

plt.show()